# view the data gathered

In [27]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
datapath="C:/Users/dexte/Documents/GitHub/3D-textures/"

def get_friction(trial1_forces,trial2_forces,trial3_forces,mass):
    mass=mass/1000 #convert to kg
    all_forces = trial1_forces[10:-10] + trial2_forces[10:-10] + trial3_forces[10:-10]
    avg_force = np.mean(all_forces)

    # Compute normal force (assuming gravity = 9.81 m/s²)
    normal_force = mass * 9.81

    # Return coefficient of friction
    mu = avg_force / normal_force
    return mu

In [35]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
datapath="C:/Users/dexte/Documents/GitHub/3D-textures/"

def get_friction(trial1_forces,trial2_forces,trial3_forces,mass):
    mass=mass/1000 #convert to kg
    gf_to_N = lambda arr: [f * 0.00981 for f in arr]
    all_forces = gf_to_N(trial1_forces[10:-5]) + gf_to_N(trial2_forces[10:-5]) + gf_to_N(trial3_forces[10:-5])
    avg_force = np.mean(all_forces)

    # Compute normal force (assuming gravity = 9.81 m/s²)
    normal_force = mass * 9.81

    # Return coefficient of friction
    mu = avg_force / normal_force
    return mu

## load in

In [38]:
def get_info(file):
    name=file.split("/")[-1]
    meta=name.split("_")
    texture=meta[1]
    mass=float(meta[2].replace(".csv",""))
    df=pd.read_csv(file)
    friction=get_friction(df['Trial 1 Force'],df['Trial 2 Force'],df['Trial 3 Force'],mass)
    return df, friction,texture



In [45]:
directory=datapath+"Friction/data/"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

d={"Textures":[],"Frictions":[]}

for file in files:
    df,friction,texture=get_info(directory+file)
    d["Textures"].append(texture)
    d["Frictions"].append(friction)

d=pd.DataFrame(d)
print(d)

  Textures  Frictions
0   carpet   0.099150
1     cork   0.274675
2    GFoam   0.296100
3   weight        NaN


c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
